In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)

import numpy as np

In [6]:
import sys
sys.path.insert(0, "/home/storage/hans/jax_reco/python")
from geo import closest_distance_dom_track, closest_distance_dom_track_v
from geo import light_travel_time, light_travel_time_v
from geo import z_component_closest_point_on_track, z_component_closest_point_on_track
from geo import light_travel_time_i3calculator, light_travel_time_i3calculator_v
from geo import convert_spherical_to_cartesian_direction
from geo import cherenkov_cylinder_coordinates, cherenkov_cylinder_coordinates_v

In [7]:
from icecube import dataclasses
from icecube.phys_services import I3Calculator
# can use the following functions:
#    I3Calculator.cherenkov_time
#    I3Calculator.closest_approach_distance
#    I3Calculator.closest_approach_position
# example: double I3Calculator::CherenkovTime(const I3Particle& particle, 
# const I3Position& position, const double IndexRefG, const double IndexRefP )

from icecube.icetray import I3Units

In [18]:
# from icetray
# https://github.com/icecube/icetray/blob/main/dataclasses/public/dataclasses/I3Constants.h
__n_ice_phase = 1.3195;
__n_ice_group = 1.35634;
__n_ice = __n_ice_group

# or use the values from clsim in our tables
#  'n_group': 1.32548384613875,
#  'n_phase': 1.30799291638281
__n_ice_phase_clsim = 1.30799291638281;
__n_ice_group_clsim = 1.32548384613875;
__n_ice_clsim = __n_ice_group

__theta_cherenkov = np.arccos(1/__n_ice_phase)
__c = 0.299792458 # m / ns
__c_ice = __c/__n_ice_group

# from Matti's code
# https://github.com/HansN87/gamma_mixture_photonics/blob/reconstruction/evaluation%20scripts/GetLLHs.py

_recip__speedOfLight = 3.3356409519815204
#_n__ = 1.32548384613875
#_tan__thetaC = (_n__**2.-1.)**0.5

# some comparisons point to difference in n_phase:
#print(np.tan(__theta_cherenkov), _tan__thetaC)
# 0.8608601802848123 0.8700042680210099

#print(__n_ice_phase, _n__)
# 1.3195 1.32548384613875

#print((__n_ice_phase**2.-1.)**0.5)
# 0.8608601802848125

In [9]:
# the icetray way ... ugh. ever heard the phrase "something being non-pythonic" ....
# will use these as point of comparison.

def get_icecube_track_i3particle(track_pos, track_dir):
    # track_pos: 3 component cartesian vector
    # track_dir: 2 component spherical vector (theta, phi)
    pos = dataclasses.I3Position(track_pos[0], track_pos[1], track_pos[2])
    d = dataclasses.I3Direction()
    d.set_theta_phi(track_dir[0], track_dir[1])
    pt = 0.0 # set vertex time to 0
    i3p = dataclasses.I3Particle(pos, d, pt, dataclasses.I3Particle.ParticleShape.InfiniteTrack, 100000 * I3Units.m)
    return i3p

def get_cherenkov_time(p, dom_pos):
    pos = dataclasses.I3Position(dom_pos[0] * I3Units.m, dom_pos[1] * I3Units.m, dom_pos[2] * I3Units.m)
    return I3Calculator.cherenkov_time(p, pos, __n_ice_group, __n_ice_phase)

def get_closest_approach_distance(p, dom_pos):
    pos = dataclasses.I3Position(dom_pos[0] * I3Units.m, dom_pos[1] * I3Units.m, dom_pos[2] * I3Units.m)
    return I3Calculator.closest_approach_distance(p, pos)

def get_closest_approach_position(p, dom_pos):
    pos = dataclasses.I3Position(dom_pos[0] * I3Units.m, dom_pos[1] * I3Units.m, dom_pos[2] * I3Units.m)
    return I3Calculator.closest_approach_position(p, pos)

In [19]:
def light_travel_time_eijndhoven(dom_pos, track_pos, track_dir):
    """
    eq. 3 / Fig. 1 of https://arxiv.org/pdf/0704.1706
    """
    v_a = dom_pos - track_pos

    closest_dist = closest_distance_dom_track(dom_pos, track_pos, track_dir)
    d1 = jnp.dot(v_a, track_dir)
    
    dt = _recip__speedOfLight * (d1 + closest_dist * (__n_ice_group_clsim * __n_ice_phase_clsim - 1) / jnp.sqrt(__n_ice_phase_clsim * __n_ice_phase_clsim -1))
    return dt

In [20]:
# single example:

track_pos = np.array([-10., 10., -200.])
track_theta = np.deg2rad(10.)
track_phi = np.deg2rad(2.)
track_dir_sph = np.array([track_theta, track_phi])
track_dir = np.array(convert_spherical_to_cartesian_direction(jnp.array(track_dir_sph)))

i3track = get_icecube_track_i3particle(track_pos, track_dir_sph)
dom_pos = np.array([0, 0, -150])

print(get_cherenkov_time(i3track, dom_pos)) # geo time
print(get_closest_approach_distance(i3track, dom_pos)) # closest distance to dom
print(get_closest_approach_position(i3track, dom_pos)[2]) # z component of closest position on track

201.58118244011703
10.375040484114887
-149.85830716519607


In [21]:
print(light_travel_time_i3calculator(dom_pos, track_pos, track_dir))

201.58118244011706


In [22]:
dt, s, z = cherenkov_cylinder_coordinates(dom_pos, track_pos, track_dir)
print(dt)
print(s)
print(z)

201.58118244011706
10.375040484114885
-149.85830716519607


In [23]:
print(light_travel_time(dom_pos, track_pos, track_dir)) # ~1.5 ns difference compared to icetray's I3Calculator.
print(closest_distance_dom_track(dom_pos, track_pos, track_dir))
print(z_component_closest_point_on_track(dom_pos, track_pos, track_dir))

199.94345145110827
10.375040484114885
-149.85830716519607


In [24]:
print(light_travel_time_eijndhoven(dom_pos, track_pos, track_dir))

199.95201012714827


In [234]:
# now demonstrate a vectorized example

track_pos = np.array([-10., 10., -200.])
track_theta = np.deg2rad(10.)
track_phi = np.deg2rad(2.)
track_dir_sph = np.array([track_theta, track_phi])
track_dir = np.array(convert_spherical_to_cartesian_direction(jnp.array(track_dir_sph)))

i3track = get_icecube_track_i3particle(track_pos, track_dir_sph)

n_doms = 50
dom_pos = np.random.normal(0.0, 300, (n_doms, 3))

# single call that computes everything over the n_doms matrix
dt, s, z = cherenkov_cylinder_coordinates_v(dom_pos, track_pos, track_dir)

In [237]:
# and the awkward icetray style loop
for i, d_pos in enumerate(dom_pos):
    dt_i3c = get_cherenkov_time(i3track, d_pos)
    print(f"delay time: {dt[i]}, {dt_i3c}") 

delay time: 775.516095032946, 775.516095032946
delay time: 1008.6407611362945, 1008.6407611362956
delay time: 2098.567612931028, 2098.5676129310277
delay time: 2655.678073021857, 2655.6780730218575
delay time: 3566.1456003985218, 3566.1456003985213
delay time: 1668.5422658959055, 1668.5422658959062
delay time: 2331.3123639501923, 2331.3123639501932
delay time: 2880.5486666943584, 2880.548666694359
delay time: 3755.6406023223867, 3755.6406023223867
delay time: 2895.536092737, 2895.5360927370007
delay time: 397.4963273040678, 397.49632730406796
delay time: 1737.081377824967, 1737.081377824967
delay time: 1586.1149150781494, 1586.1149150781496
delay time: -659.2080247032454, -659.2080247032454
delay time: 859.4893995205834, 859.4893995205838
delay time: 2362.541600081884, 2362.541600081884
delay time: 241.59319981284833, 241.59319981284906
delay time: 1258.981871927281, 1258.9818719272814
delay time: 1251.8239846690653, 1251.8239846690656
delay time: 1872.2280378006126, 1872.2280378006128

In [240]:
for i, d_pos in enumerate(dom_pos):
    s_i3c = get_closest_approach_distance(i3track, d_pos)
    print(f"closest approach distance: {s[i]}, {s_i3c}") 

closest approach distance: 252.2786283094839, 252.2786283094839
closest approach distance: 547.0912807405737, 547.0912807405737
closest approach distance: 774.9223130782854, 774.9223130782854
closest approach distance: 489.55568950401045, 489.55568950401045
closest approach distance: 435.5407112064571, 435.54071120645716
closest approach distance: 821.7018744042626, 821.7018744042625
closest approach distance: 599.7894163065424, 599.7894163065424
closest approach distance: 355.12949989498435, 355.12949989498424
closest approach distance: 451.78638554785067, 451.7863855478506
closest approach distance: 597.7924558368042, 597.7924558368042
closest approach distance: 301.90142213903783, 301.9014221390378
closest approach distance: 420.36928897209714, 420.36928897209714
closest approach distance: 439.6072269226629, 439.6072269226629
closest approach distance: 290.9073326019394, 290.90733260193934
closest approach distance: 242.65403780284623, 242.65403780284623
closest approach distance: 4

In [241]:
for i, d_pos in enumerate(dom_pos):
    z_i3c = get_closest_approach_position(i3track, d_pos)[2]
    print(f"closest approach point (z-component): {z[i]}, {z_i3c}") 



closest approach point (z-component): -198.94450428033008, -198.94450428033008
closest approach point (z-component): -396.4483953536279, -396.4483953536279
closest approach point (z-component): -280.48117499756705, -280.48117499756705
closest approach point (z-component): 141.79664569989149, 141.79664569989149
closest approach point (z-component): 459.3978526507559, 459.3978526507559
closest approach point (z-component): -449.7012122623454, -449.7012122623454
closest approach point (z-component): -53.55258857347468, -53.55258857347468
closest approach point (z-component): 329.6264037887747, 329.62640378877484
closest approach point (z-component): 500.66776790403753, 500.66776790403753
closest approach point (z-component): 114.83169746646871, 114.83169746646871
closest approach point (z-component): -355.3790726387541, -355.3790726387541
closest approach point (z-component): -66.9055665956665, -66.9055665956665
closest approach point (z-component): -128.85597619352478, -128.8559761935247